In [1]:
from Classes.LoadModel import BaseModel
from Classes.Predict import Prediction
from Classes.Summarize import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.layers import Dense, Flatten, Layer, Input, Dropout, Activation, Convolution2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from config import *
from BaseModels import Facenet, VGGFace
from AttModels import Age, Emotion, Gender, Race
import tensorflow as tf

/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
def load_images(file_path, att='All'):
    list_image = os.listdir(file_path)
    final_list = []
    for i in list_image[:5000]:
        if att.lower() =='emotion':
            img = tf.keras.preprocessing.image.load_img(os.path.join(file_path, i), target_size=(48,48), color_mode="grayscale")
        else:
            img = tf.keras.preprocessing.image.load_img(os.path.join(file_path, i), target_size=(224, 224))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = np.expand_dims(img_array, 0)
        final_list.append(img_array)
    return final_list

In [3]:
def top_layers_vggface1(model_base):
    model_base.trainable = False
    model = Sequential()
    model.add(model_base)
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [4]:
def top_layers_resnet507(model_base):
    model_base.trainable = False
    model = Sequential()
    model.add(model_base)
    model.add(Flatten())
    model.add(Dense(512, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(256, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(128, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(5, activation = 'softmax'))
    return model

# With NN

In [5]:
best_model_path = '/home/ubuntu/sheryl/Facial_Attributes_Detection/Weights'
image_path = '/home/ubuntu/sheryl/face_att_sheryl'
attribute_list = {'Wearing_Hat':'vggface', 'Eyeglasses':'vggface', 'Wearing_Necktie':'vggface', 'Hair_color':'ResNet50', 'Wearing_Necklace':'vggface', 'Mustache':'vggface', 'No_Beard':'vggface', 'Wearing_Earrings':'vggface', 'Age': 'age', 'Gender':'gender', 'Race':'race','Emotion':'emotion'}


In [6]:
from time import time

def return_prediction_time(att, attribute_list, image_list, batch=32):
    model_name = attribute_list[att]
    basemodel = BaseModel(model_name)
    model = basemodel.load_model(False)
    print(f'\nModel {model_name} for {att} Loaded!')
    if att.lower() not in ['race', 'gender', 'emotion', 'age', 'hair_color'] :
        model = top_layers_vggface1(model)
        print('vggface1 layers loaded')
    elif 'hair' in att.lower():
        model = top_layers_resnet507(model)
        print('resnet507 layers loaded')
    else:
        model = model 
        print(att, 'model loaded')
    print(f'\nTop layers for {att} added !')

    weight_file_name = ''     
    if att.lower() in ['race', 'gender', 'age', 'emotion']:
        weight_file_name = att.lower() + '_model_weights.h5'
    elif 'hair' in att.lower():
        weight_file_name = 'ResNet507_Hair_color.h5'
    else:
        weight_file_name = 'vggface1_' + att + '.h5'
    
    print('Loading best weights...')
    model.load_weights(os.path.join(best_model_path, weight_file_name))
    
    print('Compile Model...')
    if 'hair' not in att.lower():
        model.compile(RMSprop(lr=0.0001, decay=1e-6), loss='binary_crossentropy', metrics=["accuracy"])
    else:
        model.compile(RMSprop(lr=0.0001, decay=1e-6), loss='categorical_crossentropy', metrics=["accuracy"])       
    
    print('Starts Prediction...')
    
    time_list = []
    batch_list =[]
    if att == 'Emotion':
        start_time = time()
        for i in image_list:
            predictions = model.predict(i)
        total_time = round((time() - start_time), 2)
        time_per_image = 5000 / total_time
        time_per_image = round(time_per_image, 4)                       
        print('The prediction FPS for', att, 'with', model_name, 'model is', total_time, 'seconds')
        print(att, 'prediction time per image is', time_per_image, 'seconds')
    else: 
        start_time = time()
        predictions = model.predict(image_list, batch_size=batch)
        total_time = round((time() - start_time), 2)

        time_per_image = 5000 / total_time
        time_per_image = round(time_per_image, 2)                       
        print('The prediction time for', att, 'with', model_name, 'model is', total_time, 'second', 'with batch', batch)
        print(att, 'prediction time per image is', time_per_image, 'FPS', 'with batch', batch)
        print('\n')
    return total_time, time_per_image

In [7]:
# batch_10 =[]
# batch_32 =[]
# batch_100 = []
# batch_1000 =[]
# image_list = load_images(image_path)
# for att in list(attribute_list.keys()):
#     for batch in [10,32,100,1000]:
#         total_time, time_per_image = return_prediction_time('Hair_color', attribute_list, image_list, batch)
#         if batch == 10:
#             batch_10.append(time_per_image)
#         elif batch == 100:
#             batch_100.append(time_per_image)
#         elif batch == 32:
#             batch_32.append(time_per_image)
#         else:
#             batch_1000.append(time_per_image)

# With Embedding

In [8]:
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.models import Model

def loading_embedding(imagepath, model):
    """
    :param imagepath: path to the image folder
    :param model: model
    :param data: dataset
    :param layer_num: position of the layer starting from the end of the summary. Determine where to cut the model
    This function takes a model, cut the layer and save the embedding for each images in a dataset
    """
    list_x = []
    list_image = os.listdir(imagepath)
    for imgs in list_image[:5000]:
        img = image.load_img(os.path.join(imagepath, imgs), target_size=(160, 160))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = x.astype('float32') / 255.
        list_x.append(x)
    feature_x = np.vstack(list_x)
    feature = model.predict(feature_x)
    
    return feature

In [9]:
# # For embeddig 

# from time import time
# from Classes.LoadModel import BaseModel
# from Classes.Summarize import *
# from Classes.BaseCls import *
# from time import time

# pkl_model_path = '/home/ubuntu/sheryl/Facial_Attributes_Detection/pkl'
# image_path = '/home/ubuntu/sheryl/face_att_sheryl'
# # WEIGHT_PATH = '/home/ubuntu/sheryl/Facial_Attributes_Detection/Weights'
# attribute_list = ['Wearing_Hat', 'Eyeglasses', 'Wearing_Necktie', 'Wearing_Necklace', 'Mustache', 'No_Beard', 'Wearing_Earrings']
# {'Wearing_Hat':'vggface', 'Eyeglasses':'vggface', 'Wearing_Necktie':'vggface', 'Hair_color':'vggface', 'Wearing_Necklace':'vggface', 'Mustache':'vggface', 'No_Beard':'vggface', 'Wearing_Earrings':'vggface', 'Age': 'age', 'Gender':'gender', 'Race':'race','Emotion':'emotion'}
# model_name = 'facenet'
# print(f'\nLoading {model_name} Base Model...')
# basemodel = BaseModel(model_name)
# model = basemodel.load_model(True)
# model = Model(inputs=model.input, outputs=model.layers[-1].output)
# time_list_embg = []
# print('\nStarting image embedding')
# time_embedding = time()
# image_embedding  = loading_embedding(image_path, model)
# final_time_embedding = time() - time_embedding

# print('Time for image embedding is', final_time_embedding)

# time_SVC = []
# time_LR =[]
# time_XGB=[]


# for MLA in ['SVC', 'LogisticRegression', 'XGBClassifier']:
#     for att in attribute_list:
#         if MLA !='LogisticRegression':
#             pkl_filename = os.path.join(pkl_model_path, att + '_' + MLA + '.pkl')
#         else:
#             pkl_filename = os.path.join(pkl_model_path, att + 'LogisticRegression.pkl')
            
#         with open(pkl_filename, 'rb') as file:
#             clf_model = pickle.load(file)

#         print('Start prediction for', att)

#         time_prediction = time()

#         prediction = clf_model.predict(image_embedding)

#         final_time_prediction = time() - time_prediction

#         total_time = final_time_prediction + final_time_embedding
#         total_time_image = round((5000/total_time),2)
#         time_list_embg.append(total_time)
#         print('Prediction FPS', att, 'is', 5000/ final_time_prediction )
#         print('FPS for prediction and embedding', att, 'is',total_time_image) 
#         if MLA == 'SVC':
#             time_SVC.append(total_time_image)
#         elif MLA == 'LogisticRegression':
#             time_LR.append(total_time_image)
#         else:
#             time_XGB.append(total_time_image)

In [1]:
attribute_list = {'Wearing_Hat':'vggface', 'Eyeglasses':'vggface', 'Wearing_Necktie':'vggface', 'Hair_color':'vggface', 'Wearing_Necklace':'vggface', 'Mustache':'vggface', 'No_Beard':'vggface', 'Wearing_Earrings':'vggface', 'Age': 'age', 'Gender':'gender', 'Race':'race','Emotion':'emotion'}

batch_10 =[200.0,
 212.77,
 200.0,
 184.57,
 208.33,
 200.0,
 212.77,
 204.08,
 192.31,
 208.33,
 204.08,
 27.93]
batch_32 =[212.77,
 217.39,
 200.0,
 198.73,
 212.77,
 200.0,
 212.77,
 208.33,
 200.0,
 212.77,
 204.08,
 27.93]
batch_100 = [208.33,
 212.77,
 196.08,
 204.83,
 208.33,
 200.0,
 212.77,
 208.33,
 212.77,
 212.77,
 192.31,
 27.93]
batch_1000 =[222.22,
 212.77,
 212.77,
 207.38,
 212.77,
 212.77,
 208.33,
 204.08,
 217.39,
 208.33,
 208.33,
 27.93]
attribute = list(attribute_list.keys())
time_SVC = [218.19, 218.16, 208.15,'-', 194.60, 212.69, 213.34, 199.72, '-', '-', '-', '-']
time_LR = [236.93, 237.18, 237.18,'-', 237.18, 237.18, 237.18, 237.18, '-', '-', '-', '-']
time_XGB = [234.63, 236.93, 236.92,'-', 236.94, 236.93, 236.93, 236.93,'-', '-', '-', '-']
#Time for image embedding is 21.08009696006775
# FPS for image embedding 237,19

In [2]:
import pandas as pd
my_dict = {'Attribute':attribute,'Frame_per_sec(FPS) \n Batch10':batch_10,'Frame_per_sec(FPS) \n Batch32':batch_32, 'Frame_per_sec(FPS) \n Batch100':batch_100, 'Frame_per_sec(FPS) \n Batch1000':batch_1000, 'FPS_SVC':time_SVC, 'FPS_LR':time_LR, 'FPS_XGB':time_XGB }
df = pd.DataFrame(my_dict).set_index('Attribute')
df

,Frame_per_sec(FPS) Batch10,Frame_per_sec(FPS) Batch32,Frame_per_sec(FPS) Batch100,Frame_per_sec(FPS) Batch1000,FPS_SVC,FPS_LR,FPS_XGB
Attribute,,,,,,,
Wearing_Hat,200.00,212.77,208.33,222.22,218.19,236.93,234.63
Eyeglasses,212.77,217.39,212.77,212.77,218.16,237.18,236.93
Wearing_Necktie,200.00,200.00,196.08,212.77,208.15,237.18,236.92
Hair_color,184.57,198.73,204.83,207.38,-,-,-
Wearing_Necklace,208.33,212.77,208.33,212.77,194.6,237.18,236.94
Mustache,200.00,200.00,200.00,212.77,212.69,237.18,236.93
No_Beard,212.77,212.77,212.77,208.33,213.34,237.18,236.93
Wearing_Earrings,204.08,208.33,208.33,204.08,199.72,237.18,236.93
Age,192.31,200.00,212.77,217.39,-,-,-


In [ ]:
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))